In [ ]:
import folium

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

def create_data_model(points):
    # modeling....
    matrix = {}
    # Nomnre de  points passer en paramètere
    num_points = len(points)

    # Etablissons la chaine, calculons la distance eucli entre 2 paire de point pour pouvoir les classer après...
    for from_idx in range(num_points):
        # On instancie un objet pour l'instant vide dans chaque clef 
        matrix[from_idx] = {}
        for to_idx in range(num_points):

            if from_idx == to_idx:
                matrix[from_idx][to_idx] = 0
            else:
                dx = points[to_idx][0] - points[from_idx][0]
                dy = points[to_idx][1] - points[from_idx][1]
                matrix[from_idx][to_idx] = int((dx**2 + dy**2)**0.5)
    return matrix

def plot_route_on_map(df, route):
    """
    Trace la route optimisée sur une carte à l'aide de folium.

    Args:
    - df (DataFrame): le DataFrame contenant les données des arbres.
    - route (list): la liste des indices des points dans l'ordre du trajet.
    """

    # Prendre la latitude et la longitude moyennes pour centrer la carte
    mean_lat = df['geo_point_2d_a'].mean()
    mean_lon = df['geo_point_2d_b'].mean()

    # Initialiser la carte
    m = folium.Map(location=[mean_lat, mean_lon], zoom_start=14)

    # Ajouter les points à la carte
    for idx in route:
        folium.Marker(
            location=[df.iloc[idx]['geo_point_2d_a'], df.iloc[idx]['geo_point_2d_b']],
            popup=str(idx),
            icon=folium.Icon(color='blue')
        ).add_to(m)

    # Ajouter le trajet à la carte
    latlng_route = [(df.iloc[idx]['geo_point_2d_a'], df.iloc[idx]['geo_point_2d_b']) for idx in route]
    folium.PolyLine(latlng_route, color="blue", weight=2.5, opacity=1).add_to(m)

    return m

def print_solution(manager, routing, solution, df_sample):
    """
    Modifie la fonction d'impression pour inclure le tracé sur la carte.
    """

    print('Distance totale: {}'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    route = []

    while not routing.IsEnd(index):
        route.append(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
    route.append(manager.IndexToNode(index))

    # Affiche la route sur la carte
    map_display = plot_route_on_map(df_sample, route)
    display(map_display)

def main(df):
    # Prend les 500 premières lignes
    df_sample = df.head(500)

    # Création des points à partir du DataFrame (500 first lines)
    points = list(zip(df_sample['geo_point_2d_a'], df_sample['geo_point_2d_b']))
    data = create_data_model(points)

    manager = pywrapcp.RoutingIndexManager(len(data), 1, 0)
    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(from_index, to_index):
        return data[manager.IndexToNode(from_index)][manager.IndexToNode(to_index)]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Configuration de la recherche avec une limite de temps de 20 secondes.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.time_limit.seconds = 200
    search_parameters.log_search = True


    solution = routing.SolveWithParameters(search_parameters)
    if solution:
        print_solution(manager, routing, solution, df_sample)

def print_solution(manager, routing, solution, df_sample):
    """
    Modifie la fonction d'impression pour inclure le tracé sur la carte.
    """

    print('Distance totale: {}'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    route = []

    while not routing.IsEnd(index):
        route.append(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
    route.append(manager.IndexToNode(index))

    # Affiche la route sur la carte
    map_display = plot_route_on_map(df_sample, route)
    display(map_display)

# Assurez-vous que votre DataFrame est chargé en tant que "df", puis appelez la fonction main
main(df)

In [1]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

def create_data_model(points):
    """Crée la matrice de distance."""
    matrix = {}
    num_points = len(points)
    for from_idx in range(num_points):
        matrix[from_idx] = {}
        for to_idx in range(num_points):
            if from_idx == to_idx:
                matrix[from_idx][to_idx] = 0
            else:
                dx = points[to_idx][0] - points[from_idx][0]
                dy = points[to_idx][1] - points[from_idx][1]
                matrix[from_idx][to_idx] = int((dx**2 + dy**2)**0.5)
    return matrix

def main():
    # Supposons que les points correspondent à df['geo_point_2d_a'], df['geo_point_2d_b']
    points = list(zip(df['geo_point_2d_a'], df['geo_point_2d_b']))

    data = create_data_model(points)

    # Création du problème de routage et du solveur.
    manager = pywrapcp.RoutingIndexManager(len(data), 1, 0)
    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(from_index, to_index):
        return data[manager.IndexToNode(from_index)][manager.IndexToNode(to_index)]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Définissez les coûts de chaque arête.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Configuration de la recherche.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.time_limit.seconds = 30
    search_parameters.log_search = True

    # Résolution du problème.
    solution = routing.SolveWithParameters(search_parameters)
    if solution:
        print_solution(manager, routing, solution)

def print_solution(manager, routing, solution):
    print('Distance totale: {}'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Parcours:\n'
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)

main()

NameError: name 'df' is not defined